# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import json

import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port lincoln,-34.7333,135.8667,13.94,90,99,1.30,AU,1727185511
1,1,katsina-ala,7.1667,9.2833,27.42,79,95,1.94,NG,1727185512
2,2,blackmans bay,-43.0167,147.3167,12.63,73,100,0.45,AU,1727185513
3,3,edinburgh of the seven seas,-37.0676,-12.3116,11.07,81,90,5.24,SH,1727185515
4,4,gadzhiyevo,69.2551,33.3362,5.42,89,100,12.86,RU,1727185516


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# Activate the Geo extension
hvplot.extension('bokeh')
# Configure the map plot_1
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,
    color='City', 
    size='Humidity', 
    tiles='OSM', 
    hover_cols=['City', 'Humidity']

)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter cities based on the given criteria
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]


# Drop any rows with null values
filtered_cities = filtered_cities.dropna()


# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
158,158,san patricio,28.0170,-97.5169,25.97,75,0,3.03,US,1727185694
174,174,ridgecrest,35.6225,-117.6709,21.47,21,0,1.34,US,1727185713
235,235,mahina,-17.5065,-149.4890,23.87,73,0,2.06,PF,1727185785
252,252,san jose,37.3394,-121.8950,24.69,55,0,0.00,US,1727185606
267,267,constantia,44.1833,28.6500,24.38,63,0,3.87,RO,1727185744
291,291,esqueda,30.7167,-109.5833,21.25,42,0,1.74,MX,1727185849
310,310,kununurra,-15.7667,128.7333,25.99,78,0,3.60,AU,1727185870
324,324,agropoli,40.3469,14.9966,24.05,66,0,2.99,IT,1727185888
341,341,zarghun shahr,32.8473,68.4457,25.75,11,0,1.34,AF,1727185907
365,365,sao gabriel da cachoeira,-0.1303,-67.0892,26.89,83,0,1.03,BR,1727185933


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a new DataFrame 'hotel_df' using the copy function
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
158,san patricio,US,28.0170,-97.5169,75,
174,ridgecrest,US,35.6225,-117.6709,21,
235,mahina,PF,-17.5065,-149.4890,73,
252,san jose,US,37.3394,-121.8950,55,
267,constantia,RO,44.1833,28.6500,63,
291,esqueda,MX,30.7167,-109.5833,42,
310,kununurra,AU,-15.7667,128.7333,78,
324,agropoli,IT,40.3469,14.9966,66,
341,zarghun shahr,AF,32.8473,68.4457,11,
365,sao gabriel da cachoeira,BR,-0.1303,-67.0892,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotelimport requests
radius = 10000  # Set radius in meters

# Prepare the params dictionary outside of the loop
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit to one result for simplicity
    "apiKey": geoapify_key  # Your API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']  # Get latitude from the 'Lat' column
    lng = row['Lng']  # Get longitude from the 'Lng' column

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"  # Circle around coordinates with the radius
    params["bias"] = f"proximity:{lng},{lat}"  # Bias the results towards the specified coordinates

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
san patricio - nearest hotel: No hotel found
ridgecrest - nearest hotel: Best Western China Lake Inn
mahina - nearest hotel: Motu Martin
san jose - nearest hotel: Hotel De Anza
constantia - nearest hotel: Hotel Dali
esqueda - nearest hotel: Hotel Esqueda
kununurra - nearest hotel: No hotel found
agropoli - nearest hotel: limone apartments
zarghun shahr - nearest hotel: No hotel found
sao gabriel da cachoeira - nearest hotel: Hotel Deus Me Deu
pahrump - nearest hotel: Holiday Inn Express & Suites
port hedland - nearest hotel: The Esplanade Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
158,san patricio,US,28.0170,-97.5169,75,No hotel found
174,ridgecrest,US,35.6225,-117.6709,21,Best Western China Lake Inn
235,mahina,PF,-17.5065,-149.4890,73,Motu Martin
252,san jose,US,37.3394,-121.8950,55,Hotel De Anza
267,constantia,RO,44.1833,28.6500,63,Hotel Dali
291,esqueda,MX,30.7167,-109.5833,42,Hotel Esqueda
310,kununurra,AU,-15.7667,128.7333,78,No hotel found
324,agropoli,IT,40.3469,14.9966,66,limone apartments
341,zarghun shahr,AF,32.8473,68.4457,11,No hotel found
365,sao gabriel da cachoeira,BR,-0.1303,-67.0892,83,Hotel Deus Me Deu


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# Create the city map with hover information
city_map = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',  # Size of the points based on humidity
    color='City',
    hover_cols=['City', 'Country', 'Hotel Name'],  # Hover information
    title='City Map with Hotels'
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)